<a href="https://colab.research.google.com/github/dineshkumarsarangapani/Machine_learning_IITM/blob/main/MLP/week7_mlp_graded_Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
columns = ['Feature-1',	'Feature-2',	'Feature-3', 'Feature-4',	'Feature-5',	'Feature-6',	'Feature-7', 'Feature-8',	'Feature-9', 'Feature-10',	'Target']

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('data_for_large_scale_week7').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
data = pd.DataFrame.from_records(rows, columns = columns)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
y = data['Target']
X = data.drop(['Target'], axis=1)
print(X.shape)
print(y.shape)

(90000, 10)
(90000,)


In [10]:
X_array = X.to_numpy()

In [11]:
y_array = y.to_numpy()

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.3, random_state=10)


In [34]:
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

(63000, 10) (27000, 10) (63000,) (27000,)


In [35]:
X_train = X_train.reshape(-1, 90,10)
y_train = y_train.reshape(-1,90)

In [36]:
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

(700, 90, 10) (27000, 10) (700, 90) (27000,)


In [51]:
from sklearn.linear_model import SGDRegressor

In [52]:
sgd = SGDRegressor(random_state=10)

In [53]:
import numpy as np
np.unique(y_array)

array(['-1.00E+01', '-1.00E+02', '-1.01E+00', ..., '9.98E+00', '9.98E+01',
       '9.99E+01'], dtype=object)

In [60]:

for batch in range(5):
  print(batch)
  sgd.partial_fit(X_train[batch], y_train[batch])

0
1
2
3
4


In [55]:
y_pred = sgd.predict(X_test)

In [56]:
from sklearn.metrics import mean_squared_error, r2_score

print(mean_squared_error(y_test, y_pred))

2729.4196387374377


In [57]:
print(r2_score(y_test, y_pred))

0.9284197568174307


In [58]:
sgd.intercept_

array([-7.35495575])

In [61]:
sgd.coef_

array([45.50440822, 19.86915663, 69.39071286, 47.78449783, 67.58578668,
       65.49032721, 82.46552098, 50.28368789, 28.06128261, 36.81559561])

# ---------- Part 2 ------------ #

In [135]:
url = 'https://github.com/dineshkumarsarangapani/Machine_learning_IITM/raw/main/MLP/3D_spatial_network.txt'

In [150]:
import pandas as pd

filereader = pd.read_csv(url, chunksize=20000,  iterator =True, names=['OSM_ID', 'LONGITUDE', 'LATITUDE', 'ALTITUDE'])

In [151]:
full_data = pd.concat(filereader, ignore_index=True)


In [152]:
full_data.shape

(434874, 4)

In [153]:
full_data.isna().sum()

OSM_ID       0
LONGITUDE    0
LATITUDE     0
ALTITUDE     0
dtype: int64

In [154]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor

scale = StandardScaler()
reg = SGDRegressor(random_state=10)

In [155]:
full_data = scale.fit_transform(full_data)

In [162]:
X_train = full_data[:,0:3]
X_train.shape

(434874, 3)

In [163]:
y_train = full_data[:,-1]
y_train.shape

(434874,)

In [164]:
# X_train = X_train.reshape(-1, 20000,3)
# y_train = y_train.reshape(-1,20000)

ValueError: ignored

In [158]:
reg.fit(X_train, y_train)

SGDRegressor(random_state=10)

In [143]:
# x_tr = pd.DataFrame()
# for chunk_number, chunk in enumerate(filereader):
#     # some code here, if needed
#     print(chunk_number)
#     scale.partial_fit(chunk)
#     x_tr = x_tr.append(chunk)
#     if(chunk_number == 6):
#       break
#     pass
# print(x_tr.shape)
# x_tr =scale.transform(x_tr)

0
1
2
3
4
5
6
(140000, 4)


In [144]:
# X_train = x_tr[:,0:3]
# X_train.shape

(140000, 3)

In [145]:
# y_train = x_tr[:,-1]
# y_train.shape

(140000,)

In [146]:
# reg.fit(X_train, y_train)

SGDRegressor(random_state=10)

In [159]:
reg.intercept_

array([4.41874378e-05])

In [160]:
reg.coef_

array([ 0.10302726,  0.14703122, -0.1764113 ])

# Part 3 A

In [165]:
from sklearn.datasets import load_iris
data = load_iris()

In [166]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [170]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()
X = norm.fit_transform(data.data)

In [171]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, data.target, test_size=0.2, random_state=10)

In [175]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
for k in [3]:
  neigh = KNeighborsClassifier(n_neighbors=k)
  neigh.fit(X_train, Y_train)
  y_pred = neigh.predict(X_test)
  print(accuracy_score(Y_test, y_pred))
  print(f1_score(Y_test, y_pred, average='weighted'))

0.9666666666666667
0.9671111111111111
